<a href="https://colab.research.google.com/github/zhongyanmin/Gemini-API/blob/main/function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google import genai
from google.genai import types

In [ ]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
schedule_meeting_function = {
    "name": "schedule_meeting",
    "description": "Schedules a meeting with specified attendees at a given time and date.",
    "parameters": {
        "type": "object",
        "properties": {
            "attendees": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of people attending the meeting.",
            },
            "date": {
                "type": "string",
                "description": "Date of the meeting (e.g., '2024-07-29')",
            },
            "time": {
                "type": "string",
                "description": "Time of the meeting (e.g., '15:00')",
            },
            "topic": {
                "type": "string",
                "description": "The subject or topic of the meeting.",
            },
        },
        "required": ["attendees", "date", "time", "topic"],
    },
}

# Configure the client and tools
client = genai.Client(api_key=GEMINI_API_KEY)
tools = types.Tool(function_declarations=[schedule_meeting_function])
config = types.GenerateContentConfig(tools=[tools])

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Schedule a meeting with Bob and Alice for 03/14/2025 at 10:00 AM about the Q3 planning.",
    config=config,
)

# Check for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    #  In a real app, you would call your function here:
    #  result = schedule_meeting(**function_call.args)
else:
    print("No function call found in the response.")
    print(response.text)

Function to call: schedule_meeting
Arguments: {'topic': 'Q3 planning', 'attendees': ['Bob', 'Alice'], 'time': '10:00', 'date': '2025-03-14'}


Step 1: Define a function declaration

Define a function and its declaration within your application code that allows users to set light values and make an API request. This function could call external services or APIs.

In [ ]:
# Define a function that the model can call to control smart lights
set_light_values_declaration = {
    "name": "set_light_values",
    "description": "Sets the brightness and color temperature of a light.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "integer",
                "description": "Light level from 0 to 100. Zero is off and 100 is full brightness",
            },
            "color_temp": {
                "type": "string",
                "enum": ["daylight", "cool", "warm"],
                "description": "Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.",
            },
        },
        "required": ["brightness", "color_temp"],
    },
}

# This is the actual function that would be called based on the model's suggestion
def set_light_values(brightness: int, color_temp: str) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {"brightness": brightness, "colorTemperature": color_temp}

Step 2: Call the model with function declarations

Once you have defined your function declarations, you can prompt the model to use them. It analyzes the prompt and function declarations and decides whether to respond directly or to call a function. If a function is called, the response object will contain a function call suggestion.

In [ ]:
tools = types.Tool(function_declarations=[set_light_values_declaration])
config = types.GenerateContentConfig(tools=[tools])

# Define user prompt
contents = [
    types.Content(
        role="user", parts=[types.Part(text="Turn the lights down to a romantic level")]
    )
]

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config,

)

print(response.candidates[0].content.parts[0].function_call)

id=None args={'color_temp': 'warm', 'brightness': 20} name='set_light_values'


In [ ]:
import base64

part = response.candidates[0].content.parts[0]
if part.thought_signature:
  print(base64.b64encode(part.thought_signature).decode("utf-8"))

CscCAVSoXO6fEYlblX05H/WuRx6BfU1ccbYkDjD2DQEBMhiW5noS6MS1280pKJJHruG1oB66+j/m8mnyFHVPgpNKJP09pGH2dUdoe2C2maDeBfLNDK/lEWfDlB56WWwJpOEVlIjI96lEpZpGmxrP5U8D44fT+LEvZMT0d2u7GWNBuAY2R1vHrEYHzv1SlH0vs+Y+0v9X+B4rwtZZmNsqnYwZ/mHycw+KLGrR7n6jcW4/uPo9KGCD2W9P5S7p6xqjlnuYv4uwgjie1O7mZ+93v84+Tqp/JzpWCXIG2hlQD4V0qiNUEAg8Pokzn9W02HW5bdXvBfND7D8QZq903hhJf/Nr0KvP/iSthrLR2n8U/i41lJHzoCv3g0esiYUOROQ/gob8TPk9JASk1rk694yEO2TjQxUZdc7ECPAmGMvYgukyteW7SkGGyAdH


Step 3: Execute set_light_values function code

Extract the function call details from the model's response, parse the arguments , and execute the set_light_values function.

In [ ]:
# Extract tool call details, it may not be in the first part.
tool_call = response.candidates[0].content.parts[0].function_call

if tool_call.name == "set_light_values":
    result = set_light_values(**tool_call.args)
    print(f"Function execution result: {result}")

Function execution result: {'brightness': 20, 'colorTemperature': 'warm'}


In [ ]:
print(response.candidates[0].content)

parts=[Part(
  function_call=FunctionCall(
    args={
      'brightness': 20,
      'color_temp': 'warm'
    },
    name='set_light_values'
  ),
  thought_signature=b'\n\xc7\x02\x01T\xa8\\\xee\x9f\x11\x89[\x95}9\x1f\xf5\xaeG\x1e\x81}M\\q\xb6$\x0e0\xf6\r\x01\x012\x18\x96\xe6z\x12\xe8\xc4\xb5\xdb\xcd)(\x92G\xae\xe1\xb5\xa0\x1e\xba\xfa?\xe6\xf2i\xf2\x14uO\x82\x93J$\xfd=\xa4a\xf6uGh{`\xb6\x99\xa0\xde\x05\xf2\xcd\x0c\xaf\xe5\x11g\xc3\x94\x1ezYl\t\xa4...'
)] role='model'


Step 4: Create user friendly response with function result and call the model again

Finally, send the result of the function execution back to the model so it can incorporate this information into its final response to the user.

In [ ]:
# Create a function response part
function_response_part = types.Part.from_function_response(
    name=tool_call.name,
    response={"result": result},
)

# Append function call and result of the function execution to contents
contents.append(response.candidates[0].content) # Append the content from the model's response.
contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response

final_response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=config,
    contents=contents,
)

print(final_response.text)

The lights are now set to a romantic level.



In [ ]:
power_disco_ball = {
    "name": "power_disco_ball",
    "description": "Powers the spinning disco ball.",
    "parameters": {
        "type": "object",
        "properties": {
            "power": {
                "type": "boolean",
                "description": "Whether to turn the disco ball on or off.",
            }
        },
        "required": ["power"],
    },
}

start_music = {
    "name": "start_music",
    "description": "Play some music matching the specified parameters.",
    "parameters": {
        "type": "object",
        "properties": {
            "energetic": {
                "type": "boolean",
                "description": "Whether the music is energetic or not.",
            },
            "loud": {
                "type": "boolean",
                "description": "Whether the music is loud or not.",
            },
        },
        "required": ["energetic", "loud"],
    },
}

dim_lights = {
    "name": "dim_lights",
    "description": "Dim the lights.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "number",
                "description": "The brightness of the lights, 0.0 is off, 1.0 is full.",
            }
        },
        "required": ["brightness"],
    },
}

In [ ]:

# Configure the client and tools
# client = genai.Client()
house_tools = [
    types.Tool(function_declarations=[power_disco_ball, start_music, dim_lights])
]
config = types.GenerateContentConfig(
    tools=house_tools,
    automatic_function_calling=types.AutomaticFunctionCallingConfig(
        disable=True
    ),
    # Force the model to call 'any' function, instead of chatting.
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode='ANY')
    ),
)

chat = client.chats.create(model="gemini-2.5-flash", config=config)
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call
print("Example 1: Forced function calling")
for fn in response.function_calls:
    args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
    print(f"{fn.name}({args})")

Example 1: Forced function calling
start_music(loud=True, energetic=True)
power_disco_ball(power=True)
dim_lights(brightness=0.5)


In [ ]:
# Actual function implementations
def power_disco_ball_impl(power: bool) -> dict:
    """Powers the spinning disco ball.

    Args:
        power: Whether to turn the disco ball on or off.

    Returns:
        A status dictionary indicating the current state.
    """
    return {"status": f"Disco ball powered {'on' if power else 'off'}"}

def start_music_impl(energetic: bool, loud: bool) -> dict:
    """Play some music matching the specified parameters.

    Args:
        energetic: Whether the music is energetic or not.
        loud: Whether the music is loud or not.

    Returns:
        A dictionary containing the music settings.
    """
    music_type = "energetic" if energetic else "chill"
    volume = "loud" if loud else "quiet"
    return {"music_type": music_type, "volume": volume}

def dim_lights_impl(brightness: float) -> dict:
    """Dim the lights.

    Args:
        brightness: The brightness of the lights, 0.0 is off, 1.0 is full.

    Returns:
        A dictionary containing the new brightness setting.
    """
    return {"brightness": brightness}

# Configure the client
# client = genai.Client()
config = types.GenerateContentConfig(
    tools=[power_disco_ball_impl, start_music_impl, dim_lights_impl]
)

# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Do everything you need to this place into party!",
    config=config,
)

print("\nExample 2: Automatic function calling")
print(response.text)
# I've turned on the disco ball, started playing loud and energetic music, and dimmed the lights to 50% brightness. Let's get this party started!


Example 2: Automatic function calling
Alright, the disco ball is on, the music is pumping loud and energetic, and the lights are dimmed to a perfect party ambiance! Let's get this party started!


In [ ]:
# 必要なライブラリをインストールします
# Google Colabの環境では、通常すでにインストールされていますが、
# 最新版に更新するために、以下の行を有効にします。
!pip install -q -U google-generativeai

import time
import google.generativeai as genai
import json
from IPython.display import Markdown, display

# ----------------------------------------------------
# ステップ1: APIキーの設定
# ----------------------------------------------------
# セキュリティのため、Colabの「シークレット」機能を使用することを強く推奨します。
# 左側のパネルにある「鍵」アイコンをクリックし、「新しいシークレットを追加」で
# 名前: GOOGLE_API_KEY
# 値: あなたのGemini APIキー
# と設定してください。

# シークレットからAPIキーを取得
try:
    from google.colab import userdata
    genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
except ImportError:
    print("Colabのシークレット機能が使用できません。APIキーを直接設定します。")
    # 代わりに、APIキーを環境変数として設定するか、ここに直接入力することも可能です
    # genai.configure(api_key="YOUR_API_KEY")
    # 本番環境ではこの方法は避けてください。

# ----------------------------------------------------
# ステップ2: ツール（関数）の定義
# ----------------------------------------------------
# ライトをオンにする関数
def turn_on_the_lights():
    """家のライトをオンにします。"""
    print("💡 ライトをオンにしました。")
    return {"status": "success", "message": "ライトをオンにしました"}

# ライトをオフにする関数
def turn_off_the_lights():
    """家のライトをオフにします。"""
    print("💡 ライトをオフにしました。")
    return {"status": "success", "message": "ライトをオフにしました"}

# ----------------------------------------------------
# ステップ3: ツール（関数のスキーマ）をGeminiに宣言
# ----------------------------------------------------
# `genai.Tool.from_function`を使って、Python関数から直接ツールを定義します。
# turn_on_the_lights_tool = genai.Tool.from_function(turn_on_the_lights)
# turn_off_the_lights_tool = genai.Tool.from_function(turn_off_the_lights)

# ツールリスト
# tools = [turn_on_the_lights_tool, turn_off_the_lights_tool]
tools = [turn_on_the_lights, turn_off_the_lights]

# ----------------------------------------------------
# ステップ4: Geminiモデルの初期化と対話の実行
# ----------------------------------------------------
# 'gemini-2.5-flash'モデルを、定義したツールを使って初期化します。
model = genai.GenerativeModel('gemini-2.5-flash', tools=tools)

# プロンプトの定義
prompt = "ライトをオンにして、10秒待ってからオフにして。"

# Initialize contents with the initial prompt
contents = [{"role": "user", "parts": [{"text": prompt}]}]

# 対話を開始
print("ユーザーのプロンプト:", prompt)
print("Geminiが応答を生成中...")

# モデルにプロンプトを送信
response = model.generate_content(contents=contents)

# ----------------------------------------------------
# ステップ5: モデルの応答を処理
# ----------------------------------------------------
# モデルの応答がツール呼び出しの場合
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call

    # ツール呼び出しの内容を表示
    print("\nGeminiからの応答（ツール呼び出し）:")
    print(json.dumps(function_call.to_dict(), indent=2))

    function_name = function_call.name

    if function_name == "turn_on_the_lights":
        # turn_on_the_lights関数を実行
        result = turn_on_the_lights()

        # 10秒待つ
        print("⏳ 10秒間待機します...")
        time.sleep(10)

        # Next conversation turn: feed the result back to Gemini
        # This part prompts Gemini to call "turn_off_the_lights"
        print("💡 10 seconds have passed. Prompting for the next action.")

        # Append the model's response and the function response to the conversation history
        contents.append(response.candidates[0].content)
        contents.append({"role": "function", "parts": [{"functionResponse": {"name": "turn_on_the_lights", "response": result}}]})

        second_response = model.generate_content(
            contents=contents, # Pass the updated conversation history
            tools=[genai.Tool.from_function(turn_off_the_lights)] # Pass the tool as a list
        )

        # Here, a second tool call (to turn off) will occur
        second_function_call = second_response.candidates[0].content.parts[0].function_call
        if second_function_call.name == "turn_off_the_lights":
            # Execute the turn_off_the_lights function
            result_off = turn_off_the_lights()

            # Generate the final response from Gemini
            contents.append(second_response.candidates[0].content)
            contents.append({"role": "function", "parts": [{"functionResponse": {"name": "turn_off_the_lights", "response": result_off}}]})

            final_response = model.generate_content(
                contents=contents
            )
            print("\nGeminiからの最終的な応答:")
            display(Markdown(final_response.text))

    else:
        print(f"不明なツール呼び出し: {function_name}")
else:
    # If the response is text
    print("\nGeminiからのテキスト応答:")
    display(Markdown(response.text))

ユーザーのプロンプト: ライトをオンにして、10秒待ってからオフにして。
Geminiが応答を生成中...

Geminiからのテキスト応答:


10秒待つ、という動作はサポートしていません。ライトをオン・オフすることのみ実行できますが、いかがいたしますか？